In [1]:
# imports
import osmnx as ox
import numpy as np
import pyvista as pv
from shapely.geometry import Polygon, MultiPolygon
from pathlib import Path
import random


In [2]:
# extract OSM data for New Delhi

def extract_osm_data(location, radius = 500):

    # extract center point for the location
    center_point = ox.geocoder.geocode(location)

    # retrieve buildings based on center point, radius and tags
    buildings = ox.features_from_point(
        center_point,
        tags = {'building': True},
        dist = radius
    )

    # retrieve streets based on center point, radius and network type
    streets = ox.graph_from_point(
        center_point,
        dist = radius, 
        network_type = 'drive',
        simplify = False
    )

    # convert buildings to GeoDataFrame and set CRS (Coordinate Reference System)
    buildings = buildings.to_crs(epsg = 7760)

    # convert streets to GeoDataFrame and set CRS
    streets = ox.project_graph(streets, to_crs = 'epsg:7760')
    print(f"Downloaded {len(buildings)} buildings and {len(streets.edges())} street segments.")

    return buildings, streets


In [3]:
geodata = extract_osm_data("New Delhi, India", radius=500)

Downloaded 106 buildings and 312 street segments.


In [4]:
geodata[0].columns

Index(['geometry', 'building', 'name', 'nohousenumber', 'alt_name:ar',
       'alt_name:ks', 'alt_name:ur', 'internet_access', 'long_name',
       'long_name:ur', 'name:ar', 'name:az-Arab', 'name:azb', 'name:de',
       'name:el', 'name:es', 'name:grc', 'name:hi', 'name:hy', 'name:hyw',
       'name:ks', 'name:ml', 'name:pa', 'name:ug', 'name:ur', 'network',
       'operator', 'public_transport', 'railway', 'ref', 'train', 'wikipedia',
       'wikipedia:ur', 'amenity', 'name:ru', 'building:levels', 'roof:shape',
       'building:colour', 'name:en', 'building:levels:underground', 'historic',
       'tourism', 'addr:district', 'addr:housename', 'addr:state',
       'addr:street', 'landuse', 'layer', 'building:use', 'roof:colour',
       'type'],
      dtype='object')

In [5]:
# generate building footprints

def generate_footprints(buildings):

    # convert GeoDataFrame extracted from OSM into polygons

    footprints = []
    for g in buildings.geometry:
        if isinstance(g, MultiPolygon):
            # iterate through each polygon in MultiPolygon and split into individual polygons
            footprints.extend(list(g.geoms))
        elif isinstance(g, Polygon):
            # Add single polygon
            footprints.append(g)

    return footprints

In [6]:
generate_footprints(geodata[0])

[<POLYGON ((1022126.355 1001962.491, 1022117.956 1001947.292, 1022120.06 1001...>,
 <POLYGON ((1022059.221 1002135.602, 1022047.986 1002115.753, 1022034.655 100...>,
 <POLYGON ((1021801.132 1002301.261, 1021703.542 1002294.896, 1021698.753 100...>,
 <POLYGON ((1021681.909 1001797.421, 1021661.659 1001935.122, 1021681.979 100...>,
 <POLYGON ((1021448.801 1001908.242, 1021436.968 1001968.101, 1021427.468 100...>,
 <POLYGON ((1021676.68 1001949.513, 1021663.728 1001947.55, 1021636.467 10021...>,
 <POLYGON ((1022125.172 1001575.242, 1022114.399 1001558.619, 1022108.692 100...>,
 <POLYGON ((1022077.962 1001546.204, 1022094.78 1001535.662, 1022091.163 1001...>,
 <POLYGON ((1022064.304 1001561.085, 1022053.404 1001544.55, 1022047.726 1001...>,
 <POLYGON ((1022079.017 1001696.689, 1022081.747 1001700.917, 1022092.799 100...>,
 <POLYGON ((1022057.764 1001622.604, 1022047.001 1001605.982, 1022041.294 100...>,
 <POLYGON ((1022004.887 1001592.171, 1022021.704 1001581.629, 1022018.087 100...>,
 <PO

In [7]:
# create building mesh with base, walls and roof

def create_building_mesh(coords, height):

    # remove duplicate last point from polygon coordinates
    coords = coords[:-1]
    num_points = len(coords)

    # create points for base and top of building
    base_points = np.column_stack((coords, np.zeros(num_points)))
    top_points = np.column_stack((coords, np.full(num_points, height)))

    # combine base and top points
    points = np.vstack((base_points, top_points))

    # create faces

    faces = []

    # add base (as triangle fan)
    base_face = [num_points] + list(range(num_points))
    faces.extend([base_face])

    # add roof (as triangle fan)
    roof_indices = list(range(num_points, 2 * num_points))
    roof_face = [num_points] + roof_indices
    faces.extend([roof_face])

    # add walls (as quads)
    for i in range(num_points):
        next = (i+1) % num_points
        wall_face = [4, # quad face
                     i, next, # base points
                     num_points + next, num_points + i] # top points
        faces.extend(wall_face)

    return points, faces

In [8]:
# random color generation

def generate_random_colors():
    # rgb color generation
    return [random.random() for _ in range(3)]

In [9]:
# extruding buildings to create 3D mesh
def extrude_buildings(footprints):

    # extrude 2d footprints into 3D meshes

    city_mesh = pv.PolyData()
    instances_building = []
    for footprint in footprints:
        # get coordinates of the footprint
        coords = np.array(footprint.exterior.coords)

        # generate random height for the building between 10 and 50 m
        height = np.random.uniform(10, 50)

        # create building mesh
        points, faces = create_building_mesh(coords, height)
        building = pv.PolyData(points, np.hstack(faces))

        # random colors for buildings
        color = generate_random_colors()
        building['color'] = np.tile(color, (building.n_points, 1))
        instances_building.append(building)

        # add building mesh to city mesh
        if city_mesh.n_points == 0:
            city_mesh = building
        else:
            city_mesh = city_mesh.merge(building, merge_points = False)

    print("Buildings extruded and merged into city mesh.")
    return city_mesh, instances_building

In [10]:
# 3d visualization of the city mesh

location = "New Delhi, India"
radius = 1500

buildings, streets = extract_osm_data(location, radius)
footprints = generate_footprints(buildings)
city_mesh, instances_building = extrude_buildings(footprints)

p1 = pv.Plotter(border = True)
p1.add_mesh(city_mesh, scalars = 'color', rgb = True, show_edges = False)
p1.show(title = "3D City Model of New Delhi", window_size = [800, 600])

Downloaded 2368 buildings and 10308 street segments.
Buildings extruded and merged into city mesh.


Widget(value='<iframe src="http://localhost:54283/index.html?ui=P_0x137d897f0_0&reconnect=auto" class="pyvista…

In [11]:
# save 3d model
def save_city_mesh(city_mesh,output_path):

    # create output directory if it doesn't exist
    output_directory = Path(output_path).parent
    output_directory.mkdir(parents=True, exist_ok=True)

    # save the city mesh to the specified output path
    print(f"Saving city mesh to {output_path}...")
    city_mesh.save(output_path)
    print("City mesh saved successfully.")


In [12]:
def streetGraph_to_pyvista(street_graph):

    # convert graph to dataframe
    nodes, edges = ox.graph_to_gdfs(street_graph)

    # convert edges to pyvista object with lines
    points = edges['geometry'].apply(
    lambda g: np.column_stack((g.xy[0], g.xy[1], np.zeros(len(g.xy[0]))))
    ).tolist()
    
    vertices = np.concatenate(points)

    lines = []

    j = 0

    for l in range(len(points)):
        pts = points[l]
        vertex_len = len(pts)
        vertex_start, vertex_end = j, j+vertex_len-1
        vertex_array = [vertex_len] + list(range(vertex_start, vertex_end+1))
        lines.append(vertex_array)
        j += vertex_len 
    
    return pv.PolyData(vertices, np.hstack(lines))
        

In [13]:
def cloudgify(location, mesh, street_mesh, file_path):
    # using pyvista to create a gif of the 3d model
    p1 = pv.Plotter(off_screen=True, image_scale=1)
    p1.background_color = "white"
    actor = p1.add_text(
        location,
        position = 'upper_left',
        color = "lightgrey",
        shadow = True,
        font_size = 26
    )

    # building mesh with colors
    p1.add_mesh(mesh, scalars = mesh['color'], cmap = "tab20", show_edges=False)
    p1.add_mesh(street_mesh)
    p1.remove_scalar_bar()
    p1.show(auto_close=False)

    # set camera position and viewup
    viewup = [0,0,1]

    # create output directory if it doesn't exist
    output_dir = Path(file_path).parent
    output_dir.mkdir(parents=True, exist_ok=True)

    # generate orbital path and create gif
    path = p1.generate_orbital_path(n_points = 40, shift = mesh.length,
                                        viewup = viewup, factor = 3.0)
    p1.open_gif(output_dir/"model.gif")
    p1.orbit_on_path(path, write_frames = True, viewup = viewup)
    p1.close()
    print(f"GIF saved to {output_dir/'model.gif'}")
    return

In [14]:
# single location gif
location = "New Delhi, India"
radius = 1500

buildings, streets = extract_osm_data(location, radius)
footprints = generate_footprints(buildings)
city_mesh, instances_building = extrude_buildings(footprints)
street_mesh = streetGraph_to_pyvista(streets)

output_dir = "./outputs/" + location.split(",")[0]
cloudgify(location, city_mesh, street_mesh, output_dir)

output_buildings = output_dir + "buildings.obj"
output_streets = output_dir + "streets.obj"
save_city_mesh(city_mesh, output_buildings)
save_city_mesh(street_mesh, output_streets)


Downloaded 2368 buildings and 10308 street segments.
Buildings extruded and merged into city mesh.


Widget(value='<iframe src="http://localhost:54283/index.html?ui=P_0x137d23c50_1&reconnect=auto" class="pyvista…

GIF saved to outputs/model.gif
Saving city mesh to ./outputs/New Delhibuildings.obj...
City mesh saved successfully.
Saving city mesh to ./outputs/New Delhistreets.obj...
City mesh saved successfully.
